In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import urllib.robotparser

In [2]:
#Read robots.txt file
rp = urllib.robotparser.RobotFileParser()
rp.set_url("https://pureportal.coventry.ac.uk/robots.txt")
rp.read()
rrate = rp.request_rate("*")
try:
    rrate.requests
    rrate.seconds
except: 
    pass
rp.crawl_delay("*")

#Webcrawler
headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
dflist=[]
page=range(0,15)
def mycrawler(seed):
    main_url=seed+'publications/'
    for pages in page:
        seed1=main_url+'?page={}'.format(pages)
        Q = [seed1]
        count = 0
        while(Q!=[] and count < 15):
            count +=1
            url = Q.pop(0)
            code = requests.get(url,headers=headers)
            plain = code.text
            s = BeautifulSoup(plain, "html.parser")
            publications = s.find_all('div',{'class':'result-container'})
            for all_pub in publications:
                try:
                    result={
                        'title':all_pub.find('a',{'class':'link'}).text,
                        'link':all_pub.find('a',{'class':'link'})['href'],
                        'author':all_pub.find('a',{'class':'link person'}).text,
                        'author_link':all_pub.find('a',{'class':'link person'})['href'],
                        'date':all_pub.find('span',{'class':'date'}).text
                     }
                except AttributeError:
                    result={
                        'title':all_pub.find('a',{'class':'link'}).text,
                        'link':all_pub.find('a',{'class':'link'})['href'],
                        'author':None,
                        'author_link':None,
                        'date':all_pub.find('span',{'class':'date'}).text
                     }
                dflist.append(result)
                df = pd.DataFrame(dflist)
                df.to_csv('details1.csv',index=False)

                
mycrawler('https://pureportal.coventry.ac.uk/en/organisations/school-of-life-sciences/')


In [3]:
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download("punkt")
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string

[nltk_data] Downloading package punkt to C:\Users\David
[nltk_data]     Nereekshan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
#Indexer
df=pd.read_csv('details1.csv')
df['author'].fillna(df['author'].mode().tolist()[0])
df['author_link'].fillna(df['author_link'].mode().tolist()[0])
df.date=pd.to_datetime(df.date)
result = string.punctuation
output = df.replace(result,"")
sw = stopwords.words('english')
output.drop(["link", "author_link", "date"],axis=1,inplace=True)
title_list =  output.title.to_list()
author_list = output.author.to_list()
title_author = title_list+author_list
print(title_author)


['A Comparison of Physical Activity Between Home-Based and Centre-Based Pulmonary Rehabilitation: A Randomised Controlled Secondary Analysis', 'Actual and perceived motor competence mediate the relationship between physical fitness and technical skill performance in young soccer players', 'Acute Hormonal Response to Kettlebell Swing Exercise Differs Depending on Load, Even When Total Work Is Normalized', 'Associations Between Self-Reported Sleep Duration and Mortality in Employed Individuals: Systematic Review and Meta-Analysis', 'Blood cancer care in a resource limited setting during the Covid-19 outbreak; a single center experience from Sri Lanka', 'British Soccer Academy Personnel Perceive Psychological and Technical/Tactical Attributes as the Most Important Contributors to Development', 'Changes in joint kinematics and dynamic postural stability with free and restricted arm movements in children', 'Comparative analysis of reported physical activity from leisure centres’ members ver

In [5]:
string = ' '.join([str(item) for item in title_author])

In [6]:
ps = PorterStemmer()
filtered_docs = []
stem=[]
for name in string:
    tokens = word_tokenize(name)
    tmp = ""
    for w in tokens:
        if w not in sw:
            stem += ps.stem(w) + " "
            filtered_docs.append(tokens)
            print(filtered_docs)

[['A']]
[['A'], ['C']]
[['A'], ['C'], ['p']]
[['A'], ['C'], ['p'], ['r']]
[['A'], ['C'], ['p'], ['r'], ['n']]
[['A'], ['C'], ['p'], ['r'], ['n'], ['f']]
[['A'], ['C'], ['p'], ['r'], ['n'], ['f'], ['P']]
[['A'], ['C'], ['p'], ['r'], ['n'], ['f'], ['P'], ['h']]
[['A'], ['C'], ['p'], ['r'], ['n'], ['f'], ['P'], ['h'], ['c']]
[['A'], ['C'], ['p'], ['r'], ['n'], ['f'], ['P'], ['h'], ['c'], ['l']]
[['A'], ['C'], ['p'], ['r'], ['n'], ['f'], ['P'], ['h'], ['c'], ['l'], ['A']]
[['A'], ['C'], ['p'], ['r'], ['n'], ['f'], ['P'], ['h'], ['c'], ['l'], ['A'], ['c']]
[['A'], ['C'], ['p'], ['r'], ['n'], ['f'], ['P'], ['h'], ['c'], ['l'], ['A'], ['c'], ['v']]
[['A'], ['C'], ['p'], ['r'], ['n'], ['f'], ['P'], ['h'], ['c'], ['l'], ['A'], ['c'], ['v'], ['B']]
[['A'], ['C'], ['p'], ['r'], ['n'], ['f'], ['P'], ['h'], ['c'], ['l'], ['A'], ['c'], ['v'], ['B'], ['e']]
[['A'], ['C'], ['p'], ['r'], ['n'], ['f'], ['P'], ['h'], ['c'], ['l'], ['A'], ['c'], ['v'], ['B'], ['e'], ['w']]
[['A'], ['C'], ['p'], ['r'], ['n

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
string1 = ' '.join([str(item) for item in filtered_docs])
#print(string)

In [8]:
#inverted indexer
inverted_index = {}
for i,doc in enumerate(string1):
    for term in doc.split():
        if term in inverted_index:
            inverted_index[term].add(i)
        else: 
            inverted_index[term] = {i}
            #print(inverted_index)

In [9]:
def query_page(values):
    val_3 = []
    val_4 = []
    for i,doc in enumerate(title_list):
        for term in doc.split():
            if term==values:
                val_3 = i
                val_4.append(val_3)
    return val_4

In [10]:
def q_result():
    out_2 = []
    for term in out:
        var=term+1
        out_1 = df.iloc[int(term):int(var),:].values
        out_2.append(out_1)
    return out_2

In [13]:
values = input("Enter your title:\n")
out=query_page(values)
q_result()

Enter your title:
engineering


[array([['Protein engineering of Pseudomonas fluorescens peroxidase Dyp1B for oxidation of phenolic and polymeric lignin substrates',
         'https://pureportal.coventry.ac.uk/en/publications/protein-engineering-of-pseudomonas-fluorescens-peroxidase-dyp1b-f',
         'Ehibhatiomhan, A.',
         'https://pureportal.coventry.ac.uk/en/persons/austine-ehibhatiomhan',
         Timestamp('2019-04-01 00:00:00')]], dtype=object)]

In [ ]:
def textprocessing(values):
    punct = string.punctuation
    outp = df.replace(result,"")
    stw = stopwords.words('english')
    text = values.lower()
    pts = PorterStemmer()
    filtered_words = []
    stem=[]
    for inp in text:
        tkn = word_tokenize(inp)
        tmpy = ""
        for wd in tkn:
            if wd not in stw:
                stemmer += ps.stem(wd) + " "
                filtered_words.append(tkn)
                return filtered_words

In [ ]:
def vector(df):
    vectorizer = TfidfVectorizer()
    X1 = vectorizer.fit_transform(processing)
    docvectors = X1.toarray()
    return X1, docvectors

In [ ]:
def invtindex(df:list):
    invertindex = {}
    for index,doc_t in enumerate(df):
        for term in doc_t.split():
            if term in inv_idx_dict.keys():
                invertindex[term]=[index]
            elif index not in inv_idx_dict[word]: 
                invertindex[term].append(index)
    return invertindex
def run_inverted_index_test(df_sizes, df, docvectors):
    df_sizes = list(range(100, 1001, 100))

    inv_idx_run_times = []
    inv_idx_comparisons = []
    for df_size in df_sizes:
        test_vectors = docvectors[:df_size]
        test_data = df["text"].iloc[:df_size].tolist()
        num_of_comparisons = 0
        pairwise_similarity = []
        inv_idx_dict = generate_inverted_index(test_data)
        
        start = time.time()    

        for cur_doc_index, doc in enumerate(test_data):
            to_compare_indexes = [] 
            # find all the document indexes that have a common word with the current doc
            for word in doc.split():
                to_compare_indexes.extend(inv_idx_dict[word])

            # eliminate duplicates
            to_compare_indexes = list(set(to_compare_indexes))

            # calculate the similarity onlf if the id is larger than 
            # the current document id for better efficiency
            cur_doc_sims = []
            for compare_doc_index in to_compare_indexes:
                if compare_doc_index < cur_doc_index:
                    continue
                    sim = find_similarity(test_vectors[cur_doc_index], test_vectors[compare_doc_index])
                    num_of_comparisons += 1
                    cur_doc_sims.append([compare_doc_index, sim])
                    pairwise_similarity.append(cur_doc_sims)
                    end = time.time()
                    time_passed = end-start

                    inv_idx_run_times.append(time_passed)
                    inv_idx_comparisons.append(num_of_comparisons)
        
                    return inv_idx_run_times, inv_idx_comparisons, pairwise_similarity
        



In [ ]:
def postfix(infix_tokens):
    #precendence initialization
    precedence = {}
    precedence['NOT'] = 3
    precedence['AND'] = 2
    precedence['OR'] = 1
    precedence['('] = 0
    precedence[')'] = 0
    output = []
    operator_stack = []
 
 #creating postfix expression
    for token in infix_tokens:
        if (token == '('):
            operator_stack.append(token)
        elif (token == ')'):
            operator = operator_stack.pop()
            while operator != '(':
                output.append(operator)
                operator = operator_stack.pop()
                if (token in precedence):
                    if (operator_stack):
                        current_operator = operator_stack[-1]
                        while (operator_stack and precedence[current_operator] > precedence[token]):
                            output.append(operator_stack.pop())
                            if (operator_stack):
                                current_operator = operator_stack[-1]
                                operator_stack.append(token)
                            else:
                                output.append(token.lower())
                                while (operator_stack):
                                    output.append(operator_stack.pop())
                                    return output

In [ ]:
def process_query(q,dictionary_inverted):
    q = q.replace('(', '(' )
    q = q.replace(')', ')')
    q = q.split(' ')
    query = []
    for i in q:
        query.append(ps.stem(i))
        for i in range(0,len(query)):
            if ( query[i]== 'and' or query[i]== 'or' or query[i]== 'not'):
                query[i] = query[i].upper()
                results_stack = []
                postfix_queue = postfix(query)
    #evaluating postfix query expression
                for i in postfix_queue:
                        if ( i!= 'AND' and i!= 'OR' and i!= 'NOT'):
                            i = i.replace('(', ' ')
                            i = i.replace(')', ' ')
                            i = i.lower()
                            i = dictionary_inverted.get(i)
                            results_stack.append(i)
                        elif (i=='AND'):
                            a = results_stack.pop()
                            b = results_stack.pop()
                            results_stack.append(AND_op(a,b))
                        elif (i=='OR'):
                            a = results_stack.pop()
                            b = results_stack.pop()
                            results_stack.append(OR_op(a,b))
                        elif (i == 'NOT'):
                            a = results_stack.pop()
                            print(a)
                            results_stack.append(NOT_op(a))

                            return results_stack.pop()

In [ ]:
#AND two posting lists
def AND_op(word1,word2):
    if ((word1) and (word2)):
        return set(word1).intersection(word2)
    else:
        return set()
#OR two posting lists
def OR_op(word1,word2):
    return set(word1).union(word2)
 
#NOT two posting lists
def NOT_op(a):
    tot_docs = list(range(0,56))
    return set(tot_docs).symmetric_difference(a)

In [ ]:
def proximity_query(q,dictionary_positional):
    q = re.sub(r"AND", "", q)
    q = re.sub(r" ", " ", q)
    q = q.split(' ')
    query = []
    for i in q:
        query.append(ps.stem(i))
        word1 = dictionary_positional.get(query[0])
        word2 = dictionary_positional.get(query[1])
        anding = set(word1).intersection(word2)
        query[2] = re.sub(r"/", "", query[2])
        answer = []
        skip = int(query[2]) + 1
        for i in anding:
            pp1 = dictionary_positional.get(query[0])[i]
            pp2 = dictionary_positional.get(query[1])[i]
            plen1 = len(pp1)
            plen2 = len(pp2)
            ii = jj = 0 
            while ii != plen1:
                while jj != plen2:
                    if (abs(pp1[ii]-pp2[jj]) == skip):
                        answer.append(i)
                    elif pp2[jj] > pp1[ii]:
                        break 
                        jj+=1
                        ii+=1
                        answer = list(dict.fromkeys(answer))
                        return answer